In [1]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [28]:
class LiveTrade:
    
    def __init__(self):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        
        rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 24000)
        nf = pd.DataFrame(rate)


        nf['date']=pd.to_datetime(nf['time'], unit='s')

        nf.set_index(np.arange(len(nf)), inplace = True)
        nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


        self.df = nf.iloc[:500]  # First half
        self.df2 = nf.iloc[500:]  # Second half
        self.df['squeeze_on'] = False
        
        self.prev_high, self.prev_low, self.current_high, self.current_low = float(), float(), float(), float()

        self.trend_date  = None
        self.prev_trend = None
        
        self.bull_no_trend = False
        self.bear_no_trend = False

        self.no_trend_up = False
        self.no_trend_down = False
        
#     For counting 20 candles after climax

        self.buy_flag = True
        self.sell_flag = True
        self.green_count = 0
        self.red_count = 0
        
        self.df['tradable'] = 0
        
        self.result = pd.DataFrame(columns=['ENTRY DATE', 'ENTRY', 'QUANTITY', 'TRADE', 'EXIT', 'EXIT DATE', 'P/L', 'REAL P/L', 'S/L' ])
        
        self.trade_on = None
        
        self.buy_on = None
        self.sell_on = None
        
        self.past_trend()
        
    
    def past_trend(self):
        length = len(self.df)

        # Calculate max and min high values
        self.prev_high = self.df.iloc[length - 42 : length - 21]['high'].max()
        self.prev_low = self.df.iloc[length - 42 : length - 21]['low'].min()

        self.current_high = self.df.iloc[length - 21 : length]['high'].max()
        self.current_low = self.df.iloc[length - 21 : length]['low'].min()

        # Up condition
        if self.current_high > self.prev_high and self.current_low > self.prev_low:
            self.df.loc[length - 21 : length, 'trend'] = 'up'
        # Down condition
        elif self.current_high < self.prev_high and self.current_low < self.prev_low:
            self.df.loc[length - 21 : length, 'trend'] = 'down'
        else:
            self.df.loc[length - 21 : length, 'trend'] = 'no trend'

        self.prev_high = self.current_high
        self.prev_low = self.current_low
        self.trend_date = self.df['date'].iloc[-1]
        self.prev_trend = self.df['trend'].iloc[-1]
    
    
    def test_current_trend(self, high, low, l):
        
        if self.prev_trend == 'up' and self.bull_no_trend is False:
            
            if low > self.prev_low:
                self.df.loc[l, 'trend'] = 'up'
                
            else:
                self.df.loc[l, 'trend'] = 'no trend bull'
                self.bull_no_trend = True
        
        elif self.prev_trend == 'up' and self.bull_no_trend is True:
            self.df.loc[l, 'trend'] = 'no trend bull 1'
        
        elif self.prev_trend == 'down' and self.bear_no_trend is False:
            
            if high < self.prev_high:
                self.df.loc[l, 'trend'] = 'down'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend bear'
                self.bear_no_trend = True
        
        elif self.prev_trend == 'down' and self.bear_no_trend is True:
            self.df.loc[l, 'trend'] = 'no trend bear1'
        
        elif self.prev_trend == 'no trend' and self.no_trend_up is False and self.no_trend_down is False:
            
            if high > self.prev_high:
                self.df.loc[l, 'trend'] = 'up'
                self.no_trend_up = True
            
            elif low < self.prev_low:
                self.df.loc[l, 'trend'] = 'down'
                self.no_trend_down = True
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
        
        elif self.prev_trend == 'no trend' and self.no_trend_up is True:
            
            if low > self.prev_low:
                self.df.loc[l, 'trend'] = 'up'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
                self.no_trend_up = False
        
        elif self.prev_trend == 'no trend' and self.no_trend_down is True:
            
            if high < self.prev_high:
                self.df.loc[l, 'trend'] = 'down'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
                self.no_trend_down = False

    def calculate_current_trend(self, position_index, days=21):
        l = len(self.df) - 1

        if position_index + days > l:
            self.test_current_trend(self.df.iloc[l]['high'], self.df.iloc[l]['low'], l)
        
        elif position_index + days == l:
            current_high = self.df.loc[self.df.index[position_index: l + 1], 'high'].max()
            current_low = self.df.loc[self.df.index[position_index: l + 1], 'low'].min()
            
            if current_high > self.prev_high and current_low > self.prev_low:
                self.df.loc[l, 'trend'] = 'up'
            
            elif current_high < self.prev_high and current_low < self.prev_low:
                self.df.loc[l, 'trend'] = 'down'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
            
            self.trend_date = self.df['date'].iloc[-1]
            self.prev_high = current_high
            self.prev_low = current_low
            self.prev_trend = self.df['trend'].iloc[-1]
            self.bull_no_trend, self.bear_no_trend, self.no_trend_up, self.no_trend_down = [False] * 4
            
            
            
    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))

        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle.iloc[-1], kc_upper.iloc[-1], kc_lower.iloc[-1]

    def calculate_indicators(self):
        length = len(self.df) - 1

        self.df.loc[length, 'EMA_8'] = talib.EMA(self.df['close'], timeperiod=8).iloc[-1]
        self.df.loc[length, 'EMA_200'] = talib.EMA(self.df['close'], timeperiod=200).iloc[-1]
        self.df.loc[length, 'RSI'] = talib.RSI(self.df['close'], timeperiod=21).iloc[-1]

        sma = self.df['close'].rolling(window=20).mean()
        stddev = self.df['close'].rolling(window=20).std()
        lower_band = sma - (2 * stddev)
        upper_band = sma + (2 * stddev)

        kc_middle, kc_upper, kc_lower = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback=20, multiplier=1.2, atr_lookback=10)

        if (lower_band.iloc[-1] > kc_lower) and (upper_band.iloc[-1] < kc_upper):
            self.df.loc[length, 'squeeze_on'] = True
        else:
            self.df.loc[length, 'squeeze_on'] = False

        self.df.loc[length, 'EMA_20'], self.df.loc[length, 'upper_keltner'], self.df.loc[length, 'lower_keltner'] = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10)

    
    def candle(self, df):
        '''Return candle colour'''

        if df['close'] - df['open'] > 0:
            return 'G'

        else:
            return 'R'
    

    def check_body(self, df):
        #     Calculate 0.7 % of entry price
    #     fOR 1H FOREX
            one_percent = df['open']/100 
            percent = one_percent * 0.06
            hammer_per = one_percent * 0.06


        #     Calculate candle size close-open
            if df['candle'] == 'G':
                candle_size = df['close'] - df['open']
                upper_wick = df['high'] - df['close']
                lower_wick = df['open'] - df['low']

            else:
                candle_size = abs(df['close'] - df['open'])
                upper_wick = df['high'] - df['open']
                lower_wick = df['close'] - df['low']

            upper_wick_per = (upper_wick/one_percent) 
            lower_wick_per = (lower_wick/one_percent) 



        #     check for entry candle size and Max candle size 4.5 %
            if ((candle_size >= percent) and candle_size <= (4.5 * one_percent) and
             (candle_size > upper_wick) and (candle_size > lower_wick)) and (upper_wick_per < 0.06) and (lower_wick_per < 0.06):
                
                if df['candle'] == 'G' and df['high'] < df['upper_keltner'] :

                    return 1
                
                elif df['candle'] == 'R' and df['low'] > df['lower_keltner'] :

                    return 1
                
                else:
                    
                    return 0

        #     condition for hammmer
        #     lower wick is atleast 50 % of total size 
        #     Green Hammer 
        # need to add max wick of opposite dir
            elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
                  (candle_size >= hammer_per) and (candle_size > upper_wick)and
                  candle_size <= (4.5 * one_percent) and ((upper_wick_per < 0.06))  
                 and df['high'] < df['upper_keltner']):

                return 1


    #     For Red Hammer
    # need to add max wick of opposite dir
            elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
                  (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (4.5 * one_percent) and 
                  ((lower_wick_per < 0.06)) and df['low'] > df['lower_keltner'] ):

                return 1


            else:
                return 0

        
    
    def check_prev_candle(self, element):

        current_index = self.df.loc[self.df.date ==  element.date].index[0]
        current_close = element.close
        current_candle = element.candle



        if current_index > 0:
            prev_index = current_index - 1 
#           prev candle colour
            prev_candle = self.df.iloc[prev_index]['candle']

#           current candle colour
            current_candle = self.df.iloc[current_index]['candle']
            
            if current_candle == 'G':
#                 Both green current close must be higher
                if prev_candle == 'G' and self.df.iloc[current_index]['close'] > self.df.iloc[prev_index][ 'close']:
                    return 1
                
#                If last candle Red then entry candle must close above last candle open
                elif prev_candle == 'R' and self.df.iloc[current_index]['close'] > self.df.iloc[prev_index][ 'open']:
                    return 1

                else:
                    return 0
                
            elif current_candle == 'R':
#                 Both red current close must be lower
                if prev_candle == 'R' and self.df.iloc[current_index]['close'] < self.df.iloc[prev_index][ 'close']:
                    return 1
                
#                If last candle green then entry candle must close below last candle open
                elif prev_candle == 'G' and self.df.iloc[current_index]['close'] < self.df.iloc[prev_index][ 'open']:
                    return 1

                else:
                    return 0
                
    #     How to check for the 1st candle?
        return 0
    
    
#   SL Method

    def set_sl(self, df):
        '''Set SL 1% above/below to entry candle'''
        
        if df['pos'] == '1':
            
            candle_size = df['high'] - df['low']
            
            if df['candle'] == 'G':
                
                sl = df['low'] - (candle_size/100)
            
            elif df['candle'] == 'R':
                
                sl = df['high'] + (candle_size/100)
                
            return sl
        
        else:
            return 0
        
    
    def check_rsi(self, df):
        '''To check RSI and EMAs'''
        
#         ONLY TRADE WHEN 20 EMA IN FAVOUR

        if (df['candle'] == 'G' and df['RSI'] <= 68 and df['close'] > df['EMA_200'] and 
            df['trend'] == 'up' and df['EMA_20'] > df['EMA_200'] ):
            
                return 1
            
            
        elif (df['candle'] == 'R' and df['RSI'] >= 32 and df['close'] < df['EMA_200'] and 
              df['trend'] == 'down' and df['EMA_20'] < df['EMA_200'] ):
        
                return 1
        
 
        else:
            
            return 0
            
            
# To mark trades as Buy/ Sell

    def check_trade(self, df):
        
        if df['candle'] == 'G':
            
            return 'B'
        
        else:
            
            return 'S'
    
    def check_tradable(self, row):
        one_percent = row['open'] / 100
        
        length = len(self.df) -1

        # When green candle goes outside channel
        if (((row['candle'] == 'G' and row['open'] > row['upper_keltner']) or
             (row['candle'] == 'R' and row['close'] > row['upper_keltner'])) and
                self.sell_flag == True and
                (abs(row['close'] - row['upper_keltner']) / one_percent) >= 0.07):
            self.buy_flag = False
            self.green_count = 0
            self.df.at[length, 'tradable'] = 'First G Break'

        # When price remains outside after green breakout
        elif (row['close'] > row['upper_keltner'] and self.sell_flag == True and
              self.buy_flag == False and self.green_count == 0):
            self.df.at[length, 'tradable'] = 'Outside after G Break'

        # Any candle in 20 count closes outside the channel, restart the count
        elif row['close'] > row['upper_keltner'] and self.buy_flag == True and self.green_count > 0:
            self.green_count = 0
            self.buy_flag = False
            self.df.at[length, 'tradable'] = 'Outside after G Break'

        # When the price returns inside the channel after a buy breakout
        elif row['close'] < row['upper_keltner'] and self.buy_flag == False:
            self.buy_flag = True
            self.green_count = 1
            self.df.at[length, 'tradable'] = 'G count 1'

        # Counting 20 candles when green breakout moves inside the channel
        elif 0 < self.green_count < 19 and row['close'] < row['upper_keltner']:
            self.green_count += 1
            self.df.at[length, 'tradable'] = 'Counting after G Break'

        # When 20 candles after climax are over
        elif self.green_count == 19 and row['close'] < row['upper_keltner'] and self.red_count == 0:
            self.green_count = 0
            self.df.at[length, 'tradable'] = 0

        # When red candle goes outside the channel
        if (((row['candle'] == 'R' and row['open'] < row['lower_keltner']) or
             (row['candle'] == 'G' and row['close'] < row['lower_keltner'])) and
                self.buy_flag == True and
                (abs(row['lower_keltner'] - row['close']) / one_percent) >= 0.07):
            self.sell_flag = False
            self.red_count = 0
            self.df.at[length, 'tradable'] = 'Red Breakout'

        # When the price remains outside the channel after a red breakout
        elif row['close'] < row['lower_keltner'] and self.buy_flag == True and self.sell_flag == False:
            self.df.at[length, 'tradable'] = 'Outside after Red Breakout'

        # Any candle in 20 count closes outside the channel, restart the count
        elif row['close'] < row['lower_keltner'] and self.sell_flag == True and self.red_count > 0:
            self.red_count = 0
            self.sell_flag = False
            self.df.at[length, 'tradable'] = 'Outside after R Break'

        # When the price returns inside the channel after a sell breakout
        elif row['close'] > row['lower_keltner'] and self.sell_flag == False:
            self.sell_flag = True
            self.red_count = 1
            self.df.at[length, 'tradable'] = 'Sell count 1'

        # Counting 20 candles when red breakout moves inside the channel
        elif 0 < self.red_count < 19 and row['close'] > row['lower_keltner']:
            self.red_count += 1
            self.df.at[length, 'tradable'] = 'Counting after sell breakout'

        # When 20 candles after climax are over
        elif self.red_count == 19 and row['close'] > row['lower_keltner'] and self.green_count == 0:
            self.red_count = 0
            self.df.at[length, 'tradable'] = 0
            
#         When Its a normal candle
        else:
            
            self.df.at[length, 'tradable'] = 0

#   Check if candle inside Keltner's Channel
    
    def check_inside_kt(self, x):
        
        if  ( x['high'] < x['upper_keltner'] and x['low'] > x['lower_keltner']):
            return 1
        
        else:
            return 0
        
#   Apply all the conditions on current candle

    def check_conditions(self):
        
        length = len(self.df) - 1
        
#         print('In check_condition')
        
        self.df.loc[length, 'pos'] = self.check_inside_kt(self.df.iloc[-1])
        
        self.df.loc[length, 'candle'] = self.candle(self.df.iloc[-1])
        
#         Only check other condition when prev pass

        if (self.df.loc[length, 'pos'] == 1 and self.df.loc[length, 'squeeze_on'] == False and 
            self.df.loc[length, 'trend'] != 'no trend' and self.trade_on == False):
        
#             print('I am In')
            
#             Check Trade
            self.df.loc[length, 'trade'] = self.check_trade(self.df.iloc[-1])
            
            
#             Check RSI
            self.df.loc[length, 'RSI'] = self.check_rsi(self.df.iloc[-1])
            
            print('RSI: ',self.df.loc[length, 'RSI'])
    
#             check_body and check_prev_candle
            
            self.df.loc[length, 'candle_size'] = self.check_body(self.df.iloc[-1])
        
            self.df.loc[length, 'prev_candle'] = self.check_prev_candle(self.df.iloc[-1])
            
#             check_tradable

            self.check_tradable(self.df.iloc[-1])
    
#             print('Tradable: ', self.df.loc[length, 'tradable'])
    
            if (self.df.loc[length, 'RSI'] == 1 and self.df.loc[length, 'candle_size'] == 1 and
               self.df.loc[length, 'prev_candle'] == 1 and self.df.loc[length, 'tradable'] == 0):
            
                print('Hellooooo')
            
                self.df.loc[length, 'sl'] = self.set_sl(self.df.iloc[-1])
                
                result_len = len(self.result)
                
                self.result.loc[result_len, 'ENTRY DATE'] = self.df.loc[length, 'date']
                self.result.loc[result_len, 'ENTRY'] = self.df.loc[length, 'close']
                self.result.loc[result_len, 'TRADE'] = self.df.loc[length, 'trade']
                self.result.loc[result_len, 'S/L'] = self.df.loc[length, 'sl']
                
                self.trade_on = True
                
                self.calculations()

    def pip_calc(self, close, sl):
#         Calculate pips between Entry & SL
        
        if str(close).index('.') >= 3:  # JPY pair
            multiplier = 0.01
        
        else:
            multiplier = 0.0001

        pips = round(abs(sl - close) / multiplier)
        return int(pips)
    
    
    def result_pip_calc(self, close, value):
#         Calculate pip value from currency price 
        
        if str(close).index('.') >= 3:  # JPY pair
            multiplier = 100
        
        else:
            multiplier = 10000
            
        pip = value * multiplier
        
        return float(pip)
    
    
    def calc_quantity(self, pips):
#         To find the number of lots to trade

        lots = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8,
                1.9, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        
#       Risk per trade 2%  of 10K $ account
        rpt = 200
    
#         print('Pips in calc_quantity: ', pips)

#         sl = 76

        quantity = []

        for i in lots:

            if (i * pips * self.one_pip_value) <= rpt:
                quantity.append(i)
        
        if quantity:

            return max(quantity)
        
        else:
            return 0
        
        
        
    
    def calculations(self):
        
        if self.buy_on == False and self.sell_on == False:
        
            l = len(self.df) - 1 
            self.sl = round(self.df.loc[l, 'sl'], 3)
            self.f = 1
            self.trail_sl = False

            self.base_candle_index = self.df.index[-1]


            self.doji_count = 0

            pips = self.pip_calc(self.df.loc[l,'close'], self.df.loc[l, 'sl'])

    #         print('Pips: ', pips)
            quantity = self.calc_quantity(pips)

            self.result.loc[count,'QUANTITY'] = quantity

            self.candles_size = abs(self.df.loc[l,'high'] - self.df.loc[l,'low'])

            if self.df.loc[l, 'trade'] =='B':

                self.buy_on = True

                self.target = round(self.df.loc[l,'close'] + 1.5 * self.df.loc[l,'close'] - self.df.loc[l,'sl']), 3)

                self.breakeven = round(self.df.loc[l,'close'] + candles_size)

                self.manage_buy()

            else:

                self.sell_on = True

                self.target = roundself.df.loc[l,'close'] - 1.5 * abs(self.df.loc[l,'close'] - self.df.loc[l,'sl']), 3)

                self.breakeven = round(self.df.loc[l,'close'] - candles_size)

                self.manage_sell()


            
    def manage_buy(self):
        
        pass
    
    def manage_sell(self):
        
        pass      
        
        
        
    
    def Caller(self):
        # Initialize an empty DataFrame to store the past data that gets dropped
        self.past_df = pd.DataFrame()
        
#         Loop for taking the live data
        for index, row in self.df2.iterrows():
            # Add live data to the df DataFrame
            self.df = self.df.append(row, ignore_index=True)

            # Delete the first row if the DataFrame has more than 202 rows
            if len(self.df) > 500:
                # Save the dropped row in past_df
                self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
                self.df = self.df.iloc[1:].reset_index(drop=True)

                
            position_index = self.df[self.df['date'] == self.trend_date].index[0]
    
#           Check trend
            self.calculate_current_trend(position_index)
            
            # Calculate indicators for the current state of df
            self.calculate_indicators()
            
#             Apply all entr conditions
            self.check_conditions()

        # Concatenate past_df and df to get the full dataset
        self.result_df = pd.concat([self.past_df, self.df], ignore_index=True)

        # Save the result_df to an Excel file
        self.result_df.to_excel('combined.xlsx')
        
        self.result.to_excel('Record_trades.xlsx')


In [29]:
ob = LiveTrade()

ob.Caller()

C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['squeeze_on'] = False
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['tradable'] = 0
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0


C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.app

RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
Hellooooo
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
Hellooooo
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0


C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)


...

RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0


C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.app

RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0


C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:543: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1636\1805666909.py:548: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.app

RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
Hellooooo
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  1.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Tradable:  0.0
RSI:  0.0
Trada

In [ ]:
#     def calc_exit(self, index : int, count):
#         '''Calculating Exit, TRG and P/L'''

#         sl = round(self.df.loc[index]['sl'], 3)
#         f = 1
#         trail_sl = False

#         base_candle_index = index
   

#         self.doji_count = 0
    
#         pips = self.pip_calc(self.df.loc[index]['close'], self.df.loc[index]['sl'])
        
# #         print('Pips: ', pips)
#         quantity = self.calc_quantity(pips)
        
#         self.result.loc[count,'QUANTITY'] = quantity
        
#         candles_size = abs(self.df.loc[index]['high'] - self.df.loc[index]['low'])
        
#             # Target 1:1.5
#         if self.result.loc[count, 'TRADE'] == 'B':
#             target = round(self.df.loc[index]['close'] + 1.5 * (self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
# #             breakeven = round((self.df.loc[index]['close'] + target)/2, 3)
#             breakeven = round(self.df.loc[index]['close'] + candles_size)
# #             breakeven = round(self.df.loc[index]['close'] + 1.5 * (self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
#     #         print('Trg: ', target)

#         else:
#             target = round(self.df.loc[index]['close'] - 1.5 * abs(self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
# #             breakeven = round((self.df.loc[index]['close'] + target)/2, 3)
#             breakeven = round(self.df.loc[index]['close'] - candles_size)
# #             breakeven = round(self.df.loc[index]['close'] - 1.5 * abs(self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
        
#         # Target 1:2
        

#         doji_counter = 0
#         doji_flag = False
        
#         # chart 0.04 % for doji
#         chart = self.df.loc[index]['open']/100 * 0.04
        
# #         For Buy trades

#         if self.result.loc[count, 'TRADE'] == 'B':
        
#             while f == 1:

#                 if index >= (len(self.df)-1):
#                     break

#                 prev_candle_size = abs(self.df.loc[index]['close'] - self.df.loc[index]['open'])

#                 # First encounter with doji
#                 if (prev_candle_size <= chart) and (doji_flag == False):
#                     doji_counter = 1
#                     doji_flag = True
#                     upper_limit = self.df.loc[index]['high']
#                     lower_limit = self.df.loc[index]['low']

#     #           Current candle check
#                 index += 1

#                 #  For Doji Check
#                 current_candle_high = self.df.loc[index]['high']
#                 current_candle_low = self.df.loc[index]['low']

#                 # if next is also doji
#                 if (doji_flag == True) and (current_candle_high <= upper_limit and current_candle_low >= lower_limit):
#                     # Exit on 3 doji's in a row
#                     if doji_counter > 2:
#                         self.result.loc[count, 'EXIT'] = round(self.df.loc[index]['close'], 3)
#                         self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
#                         self.result.loc[count, 'REASON'] = 'DOJI EXIT'
#                         f = 0
#                     doji_counter += 1

#                 # For a non doji candle after doji
#                 else:
#                     doji_flag = False
#                     doji_counter = 0
#                     upper_limit = None
#                     lower_limit = None

                
#                 current_candle = self.df.loc[index]['candle']
                
#                 if current_candle == 'G':
#                     # TRG  and Trail SL condition

#                     if self.df.loc[index]['high'] >= target:
#                         # TRG

#                         self.result.loc[count, 'EXIT'] = target
#                         self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
#                         self.result.loc[count, 'REASON'] = 'TRG'
#         #                 print('Exit: ', result.loc[count]['EXIT'])
#                         f = 0

#                     # For Breakeven
#                     elif self.df.loc[index]['high'] >= breakeven:
#                         sl = round(min(self.df.loc[base_candle_index]['close'], self.df.loc[base_candle_index]['EMA_8']), 3)
#                         trail_sl = True


#     #               For Gap Down exit
#                     elif sl >= self.df.loc[index]['open']:

#                         self.result.loc[count, 'EXIT'] =  round(self.df.loc[index]['open'], 3)
#                         self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
#                         if trail_sl:
#                             self.result.loc[count, 'REASON'] = 'TRAIL_SL'
#                             f = 0

#                         self.result.loc[count, 'REASON'] = 'SL'
#                         f = 0

#                     # Green candle low hits SL 
#                     elif sl >= self.df.loc[index]['low']:

#                         self.result.loc[count, 'EXIT'] =  sl
#                         self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
#                         if trail_sl:
#                             self.result.loc[count, 'REASON'] = 'TRAIL_SL'
#                             f = 0

#                         self.result.loc[count, 'REASON'] = 'SL'
#                         f = 0

#                 elif current_candle == 'R':
#                 # SL condition

#     #                 For Gap down
#                     if sl >= self.df.loc[index]['open']:

#                         self.result.loc[count, 'EXIT'] =  round(self.df.loc[index]['open'], 3)
#                         self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
#                         if trail_sl:
#                             self.result.loc[count, 'REASON'] = 'TRAIL_SL'
#                             f = 0
#                         self.result.loc[count, 'REASON'] = 'SL'
#                         f = 0

#     #                 For SL
#                     elif sl >= self.df.loc[index]['low']:

#                         self.result.loc[count, 'EXIT'] = sl
#                         self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
#                         if trail_sl:
#                             self.result.loc[count, 'REASON'] = 'TRAIL_SL'
#                             f = 0
#                         self.result.loc[count, 'REASON'] = 'SL'
#         #                 print('Exit: ', result.loc[count]['EXIT'])
#                         f = 0

#             #  Max wait period 11 Days till breakeven 
#             # or current candle closed above 1% to last
#             # else exit the trade
#                 prev_close_one_percent = (self.df.loc[index - 1]['close'])/100

#                 if (index > (base_candle_index + 10)) and (self.df.loc[index]['close'] < breakeven) and (self.df.loc[index]['close'] <  (self.df.loc[index - 1]['close']) + prev_close_one_percent):
#                     # print('------------------------------------------------')
#                     # print('LATE: ', index)
#                     # print('--------------------------------------------------------')
#                     self.result.loc[count, 'EXIT'] = round(self.df.loc[index]['close'], 3)
#                     self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
#                     self.result.loc[count, 'REASON'] = 'NO MOVE'
#                     f= 0